In [73]:
import pandas as pd


In [74]:
df = pd.read_csv('data.csv')
data_artist_df = pd.read_csv('data_by_artist.csv')
data_genre_df = pd.read_csv('data_by_genres.csv')
data_year_df = pd.read_csv('data_by_year.csv')
data_w_genre_df = pd.read_csv('data_w_genres.csv')

In [75]:
df.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920


In [76]:
data_artist_df.shape

(32539, 15)

In [77]:
df['artists'] = df['artists'].str.strip('[]')

In [78]:
df['artists'] = df['artists'].str.strip('"')

In [79]:
df['artists'] = df['artists'].str.strip("'")

In [80]:
df["artists"]

0                            Mamie Smith
1                  Screamin' Jay Hawkins
2                            Mamie Smith
3                        Oscar Velazquez
4                                   Mixe
                       ...              
174384    DJ Combo', 'Sander-7', 'Tony T
174385                      Alessia Cara
174386                         Roger Fly
174387                      Taylor Swift
174388                         Roger Fly
Name: artists, Length: 174389, dtype: object

In [81]:
df = df.drop(columns='release_date')


In [82]:
df.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,valence,year
0,0.991000,Mamie Smith,0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,0.0936,149.976,0.6340,1920
1,0.643000,Screamin' Jay Hawkins,0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,0.0534,86.889,0.9500,1920
2,0.993000,Mamie Smith,0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,0.1740,97.600,0.6890,1920
3,0.000173,Oscar Velazquez,0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,0.0425,127.997,0.0422,1920
4,0.295000,Mixe,0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,0.0768,122.076,0.2990,1920


In [83]:
number_cols = ['acousticness', 'danceability', 'duration_ms', 'energy', 'explicit', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
               'popularity', 'speachiness', 'tempo', 'valence', 'year']

In [87]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

cluster_pipeline = Pipeline([('scaler', StandardScaler()),('kmeans',
                                                           KMeans(n_clusters=10, n_jobs=-1))])
X = data_genre_df.select_dtypes(np.number)
cluster_pipeline.fit(X)
data_genre_df['cluster'] = cluster_pipeline.predict(X)

In [91]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne_pipeline = Pipeline([('scaler', StandardScaler()), ('tsne', TSNE(n_components=2, verbose=2))])

genre_embedding = tsne_pipeline.fit_transform(X)

projection = pd.DataFrame(columns=['x', 'y'], data=genre_embedding)
projection['genres'] = data_genre_df['genres']
projection['cluster'] = data_genre_df['cluster']

fig = px.scatter(
    projection, x='x', y='y', color='cluster', hover_data=['x', 'y', 'genres'])
fig.show()


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 3232 samples in 0.005s...
[t-SNE] Computed neighbors for 3232 samples in 0.377s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3232
[t-SNE] Computed conditional probabilities for sample 2000 / 3232
[t-SNE] Computed conditional probabilities for sample 3000 / 3232
[t-SNE] Computed conditional probabilities for sample 3232 / 3232
[t-SNE] Mean sigma: 0.789973
[t-SNE] Computed conditional probabilities in 0.172s
[t-SNE] Iteration 50: error = 82.8363876, gradient norm = 0.0292451 (50 iterations in 1.717s)
[t-SNE] Iteration 100: error = 76.4210052, gradient norm = 0.0057877 (50 iterations in 1.401s)
[t-SNE] Iteration 150: error = 76.2094421, gradient norm = 0.0007741 (50 iterations in 1.261s)
[t-SNE] Iteration 200: error = 76.1869354, gradient norm = 0.0004076 (50 iterations in 1.274s)
[t-SNE] Iteration 250: error = 76.1811600, gradient norm = 0.0002698 (50 iterations in 1.268s)
[t-SNE] KL divergence after 250 iterati

In [94]:
song_cluster_pipeline = Pipeline([('scaler', StandardScaler()),
                                  ('kmeans', KMeans(n_clusters=20,
                                                    verbose=2, n_jobs=-1))], verbose=True)
X = df.select_dtypes(np.number)
number_cols = list(X.columns)
song_cluster_pipeline.fit(X)
song_cluster_labels = song_cluster_pipeline.predict(X)
df['cluster_label'] = song_cluster_labels

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.1s
[Pipeline] ............ (step 2 of 2) Processing kmeans, total=  34.8s


In [95]:
!pip install spotipy

In [116]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id='8d1431eb8c454721b7d4d4b74e770203',
                                                           client_secret='4f7394afa9da40838dbfe3a5b4cce0f1'))

def find_song(name, year):
  song_data = defaultdict()
  results = sp.search(q= 'track: {} year: {}'.format(name,year), limit=1)
  if results['tracks']['items'] == []:
    return None

  results = results['tracks']['items'][0]
  track_id = results['id']
  audio_features = sp.audio_features(track_id)[0]

  song_data['name'] = [name]
  song_data['year'] = [year]
  song_data['explicit'] = [int(results['explicit'])]
  song_data['duration_ms'] = [results['duration_ms']]
  song_data['popularity'] = [results['popularity']]

  for key, value in audio_features.items():
    song_data[key] = value

  return pd.DataFrame(song_data)

In [144]:
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist
import difflib

def get_song_data(song, df):

  try:
    song_data = df[(df['name'] == song['name'])
                   & (df['year'] == song['year'])].iloc[0]
    return song_data
  
  except IndexError:
    return find_song(song['name'], song['year'])

def get_mean_vector(song_list, df):
  
  song_vectors = []

  for song in song_list:
    song_data = get_song_data(song, df)
    if song_data is None:
      print('Warning: {} does not exist in Spotify or in database'.format(song['name']))
      continue
    song_vector = song_data[number_cols].values
    song_vectors.append(song_vector)

  song_matrix = np.array(list(song_vectors))
  return np.mean(song_matrix, axis=0)

def flatten_dict_list(dict_list):
  flattened_dict = defaultdict()
  for key in dict_list[0].keys():
    flattened_dict[key] = []
  
  for dictionary in dict_list:
    for key, value in dictionary.items():
      flattened_dict[key].append(value)
  return flattened_dict

def recommend_songs(song_list, df=df, n_songs=10):
  metadata_cols = ['name', 'year', 'artists']
  song_dict = flatten_dict_list(song_list)

  song_center = get_mean_vector(song_list, df)
  scaler = song_cluster_pipeline.steps[0][1]
  scaled_data = scaler.transform(df[number_cols])
  scaled_song_center = scaler.transform(song_center.reshape(1,-1))
  distances = cdist(scaled_song_center, scaled_data, 'cosine')
  index = list(np.argsort(distances)[:, :n_songs][0])

  rec_songs = df.iloc[index]
  rec_songs = rec_songs[~rec_songs['name'].isin(song_dict['name'])]
  return rec_songs[metadata_cols].to_dict(orient='records')

In [167]:
recommend_songs([{'name': 'thriller',
                  'year': 1984}])

[{'artists': 'Curtis Mayfield',
  'name': "We're a Winner - Live @ Bitter End, NYC",
  'year': 1971},
 {'artists': 'Millenium',
  'name': 'My Life Domino - Live in Krakow 2009',
  'year': 2010},
 {'artists': 'Talking Heads',
  'name': "I'm Not in Love - 2004 Remaster",
  'year': 1982},
 {'artists': 'Millenium',
  'name': 'Drunken Angels - Live in Krakow 2009',
  'year': 2010},
 {'artists': 'Paul Carrack',
  'name': 'Inspire Me - Live at Bonn, Germany, 2005',
  'year': 2020},
 {'artists': 'The Cure',
  'name': 'Lullaby - Live at Wembley 07/89 - Remix 07/09',
  'year': 1989},
 {'artists': 'Bill Withers',
  'name': 'Let Us Love - Live at Carnegie Hall, New York, NY - October 1972',
  'year': 1973},
 {'artists': 'The Pharcyde', 'name': 'Officer', 'year': 1992},
 {'artists': 'Bob Marley & The Wailers',
  'name': "Jammin' - Live At The Pavillon De Paris, 1977",
  'year': 1978},
 {'artists': 'Paul Carrack',
  'name': 'Another Cup of Coffee - Live at Buxton Opera House, 2004',
  'year': 2020}]

In [173]:
user_input = input('enter a song followed by the year it came out:')
user_input = user_input.split(', ')
input_dict = {'name': user_input[0],
              'year': int(user_input[1])}
recommend_songs([input_dict])

enter a song followed by the year it came out:monster mash, 1962


[{'artists': 'Bobby "Boris" Pickett\', \'The Crypt-Kickers',
  'name': 'Monster Mash',
  'year': 1962},
 {'artists': "Cheech And Chong with Alice Bowie', 'Alice Bowie",
  'name': 'Earache My Eye',
  'year': 1978},
 {'artists': "Rayvon', 'Red Fox", 'name': 'Bashment Party', 'year': 1998},
 {'artists': 'Blossom Dearie', 'name': "I'm Hip", 'year': 1966},
 {'artists': 'Black Sheep', 'name': 'La Menage', 'year': 1991},
 {'artists': 'The Abyssinians', 'name': 'Satta Massagana', 'year': 1976},
 {'artists': 'Clarence Carter',
  'name': 'Willie and Laura Mae Jones',
  'year': 1970},
 {'artists': "Tintin', 'Tomas Bolme', 'Bert-Åke Varg",
  'name': 'Rackham den Rödes skatt, del 36',
  'year': 1974},
 {'artists': 'Beastie Boys',
  'name': 'B-Boy Bouillabaisse: Dropping Names',
  'year': 1989},
 {'artists': 'Os Originais Do Samba', 'name': 'Canto Chorado', 'year': 1969}]

[{'artists': 'Curtis Mayfield',
  'name': "We're a Winner - Live @ Bitter End, NYC",
  'year': 1971},
 {'artists': 'Millenium',
  'name': 'My Life Domino - Live in Krakow 2009',
  'year': 2010},
 {'artists': 'Talking Heads',
  'name': "I'm Not in Love - 2004 Remaster",
  'year': 1982},
 {'artists': 'Millenium',
  'name': 'Drunken Angels - Live in Krakow 2009',
  'year': 2010},
 {'artists': 'Paul Carrack',
  'name': 'Inspire Me - Live at Bonn, Germany, 2005',
  'year': 2020},
 {'artists': 'The Cure',
  'name': 'Lullaby - Live at Wembley 07/89 - Remix 07/09',
  'year': 1989},
 {'artists': 'Bill Withers',
  'name': 'Let Us Love - Live at Carnegie Hall, New York, NY - October 1972',
  'year': 1973},
 {'artists': 'The Pharcyde', 'name': 'Officer', 'year': 1992},
 {'artists': 'Bob Marley & The Wailers',
  'name': "Jammin' - Live At The Pavillon De Paris, 1977",
  'year': 1978},
 {'artists': 'Paul Carrack',
  'name': 'Another Cup of Coffee - Live at Buxton Opera House, 2004',
  'year': 2020}]